[教程地址](https://blog.csdn.net/shijie97/article/details/100203634)

In [4]:
import sys
sys.path.append('D:\Anaconda3\Lib\site-packages')
import pandas as pd

In [5]:
df = pd.DataFrame({'Country':['China','China', 'India', 'India', 'America', 'Japan', 'China', 'India'],
                   'Income':[10000, 10000, 5000, 5002, 40000, 50000, 8000, 5000],
                    'Age':[5000, 4321, 1234, 4010, 250, 250, 4500, 4321]})
print(df)

    Age  Country  Income
0  5000    China   10000
1  4321    China   10000
2  1234    India    5000
3  4010    India    5002
4   250  America   40000
5   250    Japan   50000
6  4500    China    8000
7  4321    India    5000


# Groupby

## 按照单列分组

groups属性返回一个字典，key为索引index，value为子数据帧

In [10]:
print(df.groupby('Country'))

In [9]:
print(df.groupby('Country').groups)

{'America': Int64Index([4], dtype='int64'), 'China': Int64Index([0, 1, 6], dtype='int64'), 'India': Int64Index([2, 3, 7], dtype='int64'), 'Japan': Int64Index([5], dtype='int64')}


In [6]:
# 遍历该对象输出
for index, data in df.groupby('Country'):
    print(index)
    print(data)

America
   Age  Country  Income
4  250  America   40000
China
    Age Country  Income
0  5000   China   10000
1  4321   China   10000
6  4500   China    8000
India
    Age Country  Income
2  1234   India    5000
3  4010   India    5002
7  4321   India    5000
Japan
   Age Country  Income
5  250   Japan   50000


## 按照多列分组

In [12]:
# 根据国家和收入分组
for (index1, index2), data in df.groupby(['Country', 'Income']):
    print(index1, index2)
    print(data)

America 40000
   Age  Country  Income
4  250  America   40000
China 8000
    Age Country  Income
6  4500   China    8000
China 10000
    Age Country  Income
0  5000   China   10000
1  4321   China   10000
India 5000
    Age Country  Income
2  1234   India    5000
7  4321   India    5000
India 5002
    Age Country  Income
3  4010   India    5002
Japan 50000
   Age Country  Income
5  250   Japan   50000


# aggregate 聚合

上面说到，分组之后返回多个子数据帧，如果我想知道每个子数据帧的某些列的某些信息，如方差，极差，最值等，就可以用这个聚合操作

aggregate(函数 / lambda表达式 / 函数列表 / 字典) -> return 一个数（即标量，注意这里是标量，而不是向量），然后各个子数据帧都变成一行，再合并

- （1）当传入函数的时候，这个函数的形参为每个子数据帧的所有列，可以这么理解，相当于这个函数对每个子数据帧的每一列都进行了相同操作，这个操作指的是函数体本身的内容，因此这个函数传进来的是一个列向量，这个列向量为“每个子数据帧的每一列”。返回的结果是一个数，因此，在用了这个函数之后，每个子数据帧的每一列，都会变成一个数

In [14]:
import numpy as np

def add(x):
    return np.max(x)
 
print(df.groupby('Country').agg(add))

          Age  Income
Country              
America   250   40000
China    5000   10000
India    4321    5002
Japan     250   50000


In [15]:
print(df.groupby('Country').agg(np.max))

          Age  Income
Country              
America   250   40000
China    5000   10000
India    4321    5002
Japan     250   50000


- （2）lambda表达式即匿名函数，道理和（1）相同，不再赘述。

In [17]:
print(df.groupby('Country').agg(lambda x : np.max(x)))

          Age  Income
Country              
America   250   40000
China    5000   10000
India    4321    5002
Japan     250   50000


- （3）如果传入的是函数列表，假设每个列表里面装了M个函数，代表要对每个子数据帧的每一列，进行M个操作，每个操作也是返回一个数。因此，以这种方式用了聚合函数之后，每个子数据帧的每一列，都会变成M个数。

In [18]:
print(df.groupby('Country').aggregate([np.max, np.min, np.mean]))

          Age                    Income                     
         amax  amin         mean   amax   amin          mean
Country                                                     
America   250   250   250.000000  40000  40000  40000.000000
China    5000  4321  4607.000000  10000   8000   9333.333333
India    4321  1234  3188.333333   5002   5000   5000.666667
Japan     250   250   250.000000  50000  50000  50000.000000


- （4）传入的字典，key为列名，value为函数 / 函数列表，这样做的好处是不用对每一列都进行操作，有时候我们只关心某几列，因此这样就能对某几列进行某些操作。

In [23]:
print(df.groupby('Country').aggregate({'Age' : [np.max, np.sum]}))
print(df.groupby('Country').aggregate({'Age' : [np.max, np.sum]}))

          Age       
         amax    sum
Country             
America   250    250
China    5000  13821
India    4321   9565
Japan     250    250
          Age       
         amax    sum
Country             
America   250    250
China    5000  13821
India    4321   9565
Japan     250    250


- （5）当然，因为函数传入的是列向量，我们当然可以认为的选择我们要的列向量，再进行聚合，如下，下述方法可以代替传入字典的方法，更加直观，高效。

In [24]:
print(df.groupby('Country')['Age'].aggregate(np.max))

Country
America     250
China      5000
India      4321
Japan       250
Name: Age, dtype: int64


- （6）如果只对某一个分组感兴趣，可以用get_group单独获取你想要的那个子数据帧，再对其进行操作。

In [27]:
print(df.groupby('Country').get_group('China').aggregate({'Income' : [np.mean, np.sum]}))

            Income
mean   9333.333333
sum   28000.000000


# apply
apply(函数 / lambda表达式) -> 返回标量或者列向量，然后各个子数据帧合并成新的数据帧

In [31]:
def add(x): # 这里的x指的是整个数据帧
    return x['quantity'] + x['unit price']
 
print(df.groupby('order').apply(add)) # apply的第一个形参是self，传入的是分好后的每组
 
print(df.groupby('order').apply(lambda x : x['quantity'] + x['unit price']))

# filter